##### Extraction of explain words. This will be the data which will be analysed in 'base experiments' and 'explanations in unusual places'

In [8]:
import pandas as pd
import nltk
import glob
import os
import time
from tqdm.notebook import tqdm
path = '/Users/Svesketerning/Google-Drev/experiments/database-files'
timestr = time.strftime("%Y%m%d-%H%M")

In [27]:
# Definining word_extract function - will be copied for other experiments.
def word_extract(dataframe_context,focus_words):
    result = []
    contexts = [x for x in dataframe_context.columns]
    for index in range(len(dataframe_context.index)):
        for c in contexts:
            sentences = nltk.sent_tokenize(dataframe_context[c][index])
            for sent in sentences:
                sent = sent.lower()
                overlap = [x for x in focus_words if x in sent]
                if len(overlap)>0:
                    arc_id = dataframe_context['id'][index]
                    result.append({'id':arc_id, # NOTE THAT YOU HAVE TO LOAD ARXIV DATABASE FILE AS df_arxiv!!
                                   'mainmathcat': df_arxiv.loc[df_arxiv['id'] == arc_id]['mainmathcat'].item(),
                                   'context': c, 'sentence': sent, 'focuswords': ",".join(overlap)})
    df = pd.concat([pd.DataFrame(result[i], index=['id']) for i in range(len(result))])
    df = df.set_index('id')
    return df 

def word_extract_ramos(dataframe_context,focus_words):
    result = []
    contexts = [x for x in dataframe_context.columns]
    for index in tqdm(range(len(dataframe_context.index))):
        for c in contexts:
            sentences = nltk.sent_tokenize(dataframe_context[c][index])
            for sent in sentences:
                sent = sent.lower()
                overlap = [x for x in focus_words if x in sent]
                if len(overlap)>0:
                    arc_id = dataframe_context['id'][index]
                    result.append({'id':arc_id, # NOTE THAT YOU HAVE TO LOAD ARXIV DATABASE FILE AS df_arxiv!!
                                   'mainmathcat': df_arxiv_ramos.loc[df_arxiv_ramos['id'] == arc_id]['mainmathcat'].item(),
                                   'context': c, 'sentence': sent, 'focuswords': ",".join(overlap)})
    df = pd.concat([pd.DataFrame(result[i], index=['id']) for i in range(len(result))])
    df = df.set_index('id')
    return df 

In [26]:
df_arxiv_ramos.loc[df_arxiv_ramos['id'] == '0901.0015']['mainmathcat'].item()

'math.IT'

In [20]:
# Load latest context and ArXiv database file
LatestContextFile = max(glob.iglob('/Users/Svesketerning/Google-Drev/experiments/database-files/arxiv_contexts*'),key=os.path.getctime)
df_contexts = pd.read_feather(LatestContextFile)

LatestDatabaseFile = max(glob.iglob('/Users/Svesketerning/Google-Drev/experiments/database-files/arxiv_extended_database*'),key=os.path.getctime)
df_arxiv = pd.read_feather(LatestDatabaseFile)

LatestRamosDatabaseFile = max(glob.iglob('/Users/Svesketerning/Google-Drev/experiments/database-files/ramos2009a_arxiv_extended_database*'),key=os.path.getctime)
df_arxiv_ramos = pd.read_feather(LatestRamosDatabaseFile)

explain_words = ['explanation','explain']

In [62]:
explain_raw_df = word_extract(df_contexts,explain_words)
explain_raw_df = explain_raw_df.reset_index()
explain_raw_df.to_feather(path+'/explain_fullraw'+timestr+'.feather')

#### Ramos word_df for comparison

In [28]:
LatestRamosFile = max(glob.iglob('/Users/Svesketerning/Google-Drev/experiments/database-files/ramos2009a_arxiv_contexts*'),key=os.path.getctime)
df_ramos_contexts = pd.read_feather(LatestRamosFile)

explain_ramos_raw_df = word_extract_ramos(df_ramos_contexts,explain_words)

explain_ramos_raw_df.reset_index().to_feather(path+'/explain_ramos_fullraw'+timestr+'.feather')

In [11]:
explain_full_raw_df= word_extract2(df_ramos_contexts,explain_words)